In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
# import matplotlib.pyplot as plt 
# plt.rc("font", size=14)
# import seaborn as sns
# sns.set(style="white") #white background style for seaborn plots
# sns.set(style="whitegrid", color_codes=True)

testData = pd.read_csv("test.csv")
testY = pd.read_csv("output.csv")
testData['stroke'] = testY['stroke']
trainData = pd.read_csv("train.csv")

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
trainData = trainData.drop( columns=['ever_married', 'gender'] )
testData = testData.drop( columns=['ever_married', 'gender'] )

In [ ]:
#Filling Missing BMI Data === train

temp = trainData['bmi'].mean() + trainData['bmi'].median()
temp /= 2
temp
trainData['bmi'] = trainData['bmi'].fillna(temp)

In [ ]:
#Filling Missing BMI Data ===test

temp = testData['bmi'].mean() + testData['bmi'].median()
temp /= 2
temp
testData['bmi'] = testData['bmi'].fillna(temp)

In [ ]:
print( "Empty Smoking Status(Train) in " , 
     len( list( filter( lambda x: x == True , pd.isna(trainData['smoking_status']) ) ) )
     )


print( "Empty Smoking Status(Test) in " , 
     len( list( filter( lambda x: x == True , pd.isna(testData['smoking_status']) ) ) )
     )

In [ ]:
# Segregating into Two Models

trainDataNS = trainData[ trainData['smoking_status'].isna() ]
trainDataNS = trainDataNS.drop( columns=['smoking_status'] )
trainDataS = trainData[ trainData['smoking_status'].notna() ]

testDataNS = testData[ testData['smoking_status'].isna() ]
testDataNS = testDataNS.drop( columns=['smoking_status'] )
testDataS = testData[ testData['smoking_status'].notna() ]

In [ ]:
print( "Imbalance in train is ", trainDataS['stroke'].value_counts() )
print( "Imbalance in test is ", testDataS['stroke'].value_counts() )

In [ ]:
# X and Y division

yTrainDataNS = trainDataNS['stroke']
xTrainDataNS = trainDataNS.drop( columns=['stroke'] )

yTrainDataS = trainDataS['stroke']
xTrainDataS = trainDataS.drop( columns=['stroke'] )

In [ ]:
yTestDataNS = testDataNS['stroke']
xTestDataNS = testDataNS.drop( columns=['stroke'] )

yTestDataS = testDataS['stroke']
xTestDataS = testDataS.drop( columns=['stroke'] )

In [ ]:
#Nominal Categories to one-hot encoding === Train

#Smoke Data
xTrainDataS = pd.get_dummies( xTrainDataS, 
                             columns=[ 'work_type' , 'Residence_type', 'smoking_status']
                            , prefix= ['work_type' , 'res_type', 'smoke'] )


#No Smoke Data
xTrainDataNS = pd.get_dummies( xTrainDataNS,
                              columns=[ 'work_type' , 'Residence_type']
                            , prefix=['[work_type' , 'res_type'] )



In [ ]:
#Nominal Categories to one-hot encoding ===Test

#Smoke Data
xTestDataS = pd.get_dummies( xTestDataS,
                            columns=['work_type' , 'Residence_type', 'smoking_status']
                            , prefix=['work_type' , 'res_type', 'smoke'] )


#No Smoke Data
xTestDataNS = pd.get_dummies( xTestDataNS,
                             columns=['work_type' , 'Residence_type']
                            , prefix=['work_type' , 'res_type'] )

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=0)

xTrainDataNS, yTrainDataNS = smote.fit_sample(xTrainDataNS, yTrainDataNS)
xTrainDataS, yTrainDataS = smote.fit_sample(xTrainDataS, yTrainDataS)


In [ ]:
from sklearn.model_selection import train_test_split

#Train Splitter

x_train, x_test, y_train, y_test = train_test_split(xTrainDataNS, yTrainDataNS, test_size=0.35, random_state=0)

x_trainS, x_testS, y_trainS, y_testS = train_test_split(xTrainDataS, yTrainDataS, test_size=0.35, random_state=0)

In [ ]:
#Decison Tree

from sklearn import tree
clf = tree.DecisionTreeClassifier()

#No Smoke
# clf = clf.fit(x_train, y_train)
#Smoke
clf = clf.fit(x_trainS, y_trainS)

In [ ]:
predictions = clf.predict(x_testS)
clf.score(x_testS, y_testS)

In [ ]:
from sklearn import metrics

metrics.confusion_matrix(y_testS, predictions)

In [ ]:
xTestDataS.head()

In [ ]:
#Testing
predictions = clf.predict(xTestDataS)
print(clf.score(xTestDataS, yTestDataS))

metrics.confusion_matrix(yTestDataS, predictions)

In [ ]:
###XGBOOST

from xgboost import XGBClassifier as XGBC

model = XGBC()

model.fit(x_trainS, y_trainS)


In [ ]:
predictions = model.predict(x_testS)
print(model.score(x_testS, y_testS))

metrics.confusion_matrix(y_testS, predictions)

In [ ]:
predictions = model.predict(xTestDataS.values)
print(model.score(xTestDataS.values, yTestDataS.values))

metrics.confusion_matrix(yTestDataS, predictions)

In [ ]:
##Logistic Regression

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

rfe = RFE(logreg, 20)
rfe = rfe.fit(x_trainS, y_trainS)
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
import statsmodels.api as sm
logit_model=sm.Logit(x_trainS, y_trainS)
result=logit_model.fit()
print(result.summary2())

In [ ]:
model = logreg.fit(x_trainS, y_trainS)

predictions = model.predict(x_testS)
print(model.score(x_testS, y_testS))

metrics.confusion_matrix(y_testS, predictions)

In [ ]:
predictions = model.predict(xTestDataS.values)
print(model.score(xTestDataS.values, yTestDataS.values))

metrics.confusion_matrix(yTestDataS, predictions)